In [107]:
import matplotlib.pyplot as plt
import numpy as np
import csv
from datetime import datetime, timedelta

__author__ = "Rhishabh Hattarki"
__date__ = "11 October 2023"
__assignment = "SER594: Homework 3 Q3 Programming"

In [108]:
# load csv file,
# columns is a list of all column names,
# data is a 2d list of all rows within every column
# eg if columns are A, B, C, data will be [[rows_of_A], [rows_of_B], [rows_of_C]]
def load_csv_file(input_filename):
    columns = []
    col_len = 0
    data = []
    line_count = 0
    
    with open(input_filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        
        for row in csv_reader:
            if line_count == 0:
                columns = row.copy()
                col_len = len(columns)
                data = [[] for _col in range(col_len)]
            else:
                for (i, col) in enumerate(row):
                    data[i].append(col)
            line_count += 1

    print(f'Read {line_count} lines from file {input_filename}')
    return (columns, data)

In [127]:
def clean_dates(columns, data):
    idx = columns.index('Publish_Date')
    view_idx = columns.index('Total_Views')
    dates_data = data[idx]
    cleaned_dates = []
    empty = 0
    cleaned = 0
    recorded_date = datetime(2023, 9, 23)

    for i, date_row in enumerate(dates_data):
        if date_row == '':
            if data[view_idx][i].startswith('Premieres'):
                date_format = "%m/%d/%y, %I:%M %p"
                desired_date = datetime.strptime(data[view_idx][i].lstrip('Premieres '), date_format).date()
                print(f'formatter premier date: {desired_date}')
                cleaned_dates.append(desired_date)
            else:
                empty += 1
                cleaned_dates.append(datetime.now().date())
                continue

        stream_removed = date_row
        
        if date_row.startswith('Streamed '):
            stream_removed = date_row.lstrip('Streamed ')

        converted_date = recorded_date

        if stream_removed.endswith(' months ago') or stream_removed.endswith(' month ago'):
            month_removed = stream_removed.rstrip(' months ago').rstrip(' month ago')
            converted_date = recorded_date - timedelta(days = int(month_removed) * 30)
        elif stream_removed.endswith(' year ago') or stream_removed.endswith(' years ago'):
            year_removed = stream_removed.rstrip(' year ago').rstrip(' years ago')
            converted_date = recorded_date - timedelta(days = int(year_removed) * 365)

        cleaned_dates.append(converted_date.date())
        cleaned += 1

    print(f'Cleaned {cleaned} rows in Publish_Date column and {empty} were empty')
    cleaned_data = [row.copy() for row in data]
    cleaned_data[idx] = cleaned_dates
    return columns, cleaned_data

In [128]:
def clean_views(columns, data):
    idx = columns.index('Total_Views')
    views_data = data[idx]
    cleaned_views_data = []
    empty = 0
    cleaned = 0

    for row in views_data:
        if row == '':
            empty += 1
            cleaned_views_data.append(0)
            continue
            
        unit_removed = row.rstrip(' view').rstrip(' views')
        converted_view = 0

        if unit_removed.startswith('Premieres') or unit_removed.startswith('No'):
            converted_view = 0
        elif unit_removed.endswith('K'):
            converted_view = float(unit_removed.rstrip('K')) * 1000
        elif unit_removed.endswith('M'):
            converted_view = float(unit_removed.rstrip('M')) * 1000_000
        elif unit_removed.endswith('B'):
            converted_view = float(unit_removed.rstrip('B')) * 1000_000_000
        else:
            converted_view = float(unit_removed)

        cleaned_views_data.append(int(converted_view))
        cleaned += 1

    print(f'Cleaned {cleaned} rows in Total_Views column and {empty} were empty')
    cleaned_data = [row.copy() for row in data]
    cleaned_data[idx] = cleaned_views_data
    return columns, cleaned_data

In [132]:
def preprocess_data(columns, data):
    date_cols, dates_cleaned = clean_dates(columns, data)
    print(f'\nEg after dates cleaned: {[row[0] for row in dates_cleaned]}')
    
    view_cols, views_cleaned = clean_views(date_cols, dates_cleaned)
    print(f'\nEg after views cleaned: {[row[0] for row in views_cleaned]}')

    return date_cols, dates_cleaned

In [133]:
def do_stuff(input_filename):
    """
    See homework assignment.
    :param input_filename: Filename for input datafile.
    """

    columns, data = load_csv_file(input_filename)
    prepro_cols, prepro_data = preprocess_data(columns, data)

In [134]:
if __name__ == '__main__':
    filename = "HW03_Kaggle_Youtube_DataScience.csv"
    do_stuff(filename)

Read 1559 lines from file HW03_Kaggle_Youtube_DataScience.csv
formatter premier date: 2022-09-25
Cleaned 1558 rows in Publish_Date column and 0 were empty

Eg after dates cleaned: ['Data Analyst vs Data Scientist | What are the differences?', '608 views', 'McKay Johns', 'https://www.youtube.com/watch?v=EIOt88gZWtg', 'https://www.youtube.com/c/McKayJohns', datetime.date(2022, 9, 23), "In this video, we'll cover four key differences between a data analyst and a data scientist. 1. Responsibilities 2. Education 3."]
Cleaned 1558 rows in Total_Views column and 0 were empty

Eg after views cleaned: ['Data Analyst vs Data Scientist | What are the differences?', 608, 'McKay Johns', 'https://www.youtube.com/watch?v=EIOt88gZWtg', 'https://www.youtube.com/c/McKayJohns', datetime.date(2022, 9, 23), "In this video, we'll cover four key differences between a data analyst and a data scientist. 1. Responsibilities 2. Education 3."]
